# Análise Exploratória de Dados (EDA)
## Dados Meteorológicos do INMET - Pernambuco

**Objetivo:** Realizar análise exploratória dos dados meteorológicos coletados das estações automáticas do INMET no estado de Pernambuco.

**Foco:** Entender a distribuição dos dados, identificar problemas e preparar para tratamento e modelagem.

---


## 1. Importação de Bibliotecas


In [10]:
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package.split('-')[0].split('[')[0])
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])

dependencies = [
    'pandas', 'numpy', 'matplotlib', 'seaborn', 'sqlalchemy', 'psycopg2-binary'
]

for dep in dependencies:
    install_if_missing(dep)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Bibliotecas importadas com sucesso!")


Bibliotecas importadas com sucesso!


## 2. Conexão com PostgreSQL


In [11]:
# Configurações de conexão com PostgreSQL
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'inmet_db',
    'user': 'inmet_user',
    'password': 'inmet_password'
}

print("Configuração de conexão definida!")


Configuração de conexão definida!


## 3. Carregamento dos Dados


In [12]:
# Query para carregar dados meteorológicos
query = """
SELECT 
    dm.*,
    e.nome as nome_estacao,
    e.uf,
    e.latitude,
    e.longitude
FROM dados_meteorologicos dm
JOIN estacoes e ON dm.codigo_wmo = e.codigo_wmo
ORDER BY dm.timestamp_utc
"""

# Lê dados usando conexão direta com psycopg2 (evita problemas de encoding)
try:
    conn = psycopg2.connect(
        host=DB_CONFIG['host'],
        port=DB_CONFIG['port'],
        database=DB_CONFIG['database'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password']
    )
    conn.set_client_encoding('UTF8')
    df = pd.read_sql(query, conn)
    conn.close()
    
    # Converte timestamp
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    df = pd.DataFrame()

print(f"Dados carregados: {len(df):,} registros")

# Verifica se há dados antes de processar
if len(df) == 0:
    print("\nATENÇÃO: Nenhum dado encontrado no banco!")
    print("\nPara carregar dados, execute:")
    print("1. Coloque arquivos CSV em: fastapi/app/data/raw/")
    print("2. Execute: curl -X POST http://localhost:8000/ingest")
    print("3. Execute: curl -X POST http://localhost:8000/load-to-db")
    print("\nOu use a interface do FastAPI em: http://localhost:8000/docs")
else:
    print(f"Período: {df['timestamp_utc'].min()} até {df['timestamp_utc'].max()}")
    print(f"Estações: {df['codigo_wmo'].nunique()} estações únicas")
    print("\nPrimeiras linhas:")
    display(df.head())


Dados carregados com sucesso!
Dados carregados: 0 registros

ATENÇÃO: Nenhum dado encontrado no banco!

Para carregar dados, execute:
1. Coloque arquivos CSV em: fastapi/app/data/raw/
2. Execute: curl -X POST http://localhost:8000/ingest
3. Execute: curl -X POST http://localhost:8000/load-to-db

Ou use a interface do FastAPI em: http://localhost:8000/docs


## 4. Informações Gerais do Dataset


In [13]:
if len(df) > 0:
    print("=" * 60)
    print("INFORMAÇÕES GERAIS DO DATASET")
    print("=" * 60)
    print(f"\nDimensões: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
    print(f"Número de estações: {df['codigo_wmo'].nunique()}")
    
    if pd.notna(df['timestamp_utc'].min()) and pd.notna(df['timestamp_utc'].max()):
        print(f"Período: {df['timestamp_utc'].min()} até {df['timestamp_utc'].max()}")
        total_dias = (df['timestamp_utc'].max() - df['timestamp_utc'].min()).days
        print(f"Total de dias: {total_dias} dias")
    
    print("\n" + "=" * 60)
    print("ESTAÇÕES METEOROLÓGICAS")
    print("=" * 60)
    estacoes = df.groupby('codigo_wmo').agg({
        'nome_estacao': 'first',
        'uf': 'first',
        'timestamp_utc': ['min', 'max', 'count']
    }).round(2)
    estacoes.columns = ['Estação', 'UF', 'Data Início', 'Data Fim', 'Total Registros']
    display(estacoes)
    
    print("\n" + "=" * 60)
    print("TIPOS DE DADOS")
    print("=" * 60)
    print(df.dtypes)
else:
    print("Nenhum dado disponível para análise!")


Nenhum dado disponível para análise!


## 5. Análise de Valores Faltantes


In [5]:
if len(df) > 0:
    # Análise de valores faltantes
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Variável': missing.index,
        'Valores Faltantes': missing.values,
        'Percentual (%)': missing_pct.values
    }).sort_values('Valores Faltantes', ascending=False)
    
    missing_df = missing_df[missing_df['Valores Faltantes'] > 0]
    
    print("=" * 60)
    print("VALORES FALTANTES POR VARIÁVEL")
    print("=" * 60)
    display(missing_df)
    
    # Visualização
    if len(missing_df) > 0:
        plt.figure(figsize=(12, 8))
        plt.barh(missing_df['Variável'], missing_df['Percentual (%)'])
        plt.xlabel('Percentual de Valores Faltantes (%)', fontsize=12, fontweight='bold')
        plt.title('Análise de Valores Faltantes por Variável', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    else:
        print("\nNenhum valor faltante encontrado!")
else:
    print("Nenhum dado disponível para análise!")


Nenhum dado disponível para análise!


## 6. Estatísticas Descritivas


In [6]:
if len(df) > 0:
    # Seleciona variáveis numéricas principais
    vars_numericas = [
        'precipitacao_mm', 'pressao_estacao_mb', 'temperatura_ar_c',
        'umidade_rel_horaria_pct', 'vento_velocidade_ms', 'radiacao_global_kjm2'
    ]
    
    vars_existentes = [v for v in vars_numericas if v in df.columns]
    
    if vars_existentes:
        print("=" * 60)
        print("ESTATÍSTICAS DESCRITIVAS")
        print("=" * 60)
        display(df[vars_existentes].describe())
        
        # Visualização de distribuições
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.flatten()
        
        for i, var in enumerate(vars_existentes[:6]):
            if i < len(axes):
                df[var].hist(bins=50, ax=axes[i], edgecolor='black', alpha=0.7)
                axes[i].set_title(f'Distribuição de {var}', fontweight='bold')
                axes[i].set_xlabel(var)
                axes[i].set_ylabel('Frequência')
                axes[i].grid(alpha=0.3)
        
        # Remove eixos vazios
        for i in range(len(vars_existentes), len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()
    else:
        print("Nenhuma variável numérica encontrada!")
else:
    print("Nenhum dado disponível para análise!")


Nenhum dado disponível para análise!


## 7. Análise de Precipitação (Variável Principal)


In [7]:
if len(df) > 0 and 'precipitacao_mm' in df.columns:
    print("=" * 60)
    print("ANÁLISE DE PRECIPITAÇÃO")
    print("=" * 60)
    
    # Estatísticas
    print(f"\nEstatísticas de Precipitação:")
    print(f"  Média: {df['precipitacao_mm'].mean():.2f} mm")
    print(f"  Mediana: {df['precipitacao_mm'].median():.2f} mm")
    print(f"  Máximo: {df['precipitacao_mm'].max():.2f} mm")
    print(f"  Mínimo: {df['precipitacao_mm'].min():.2f} mm")
    print(f"  Desvio Padrão: {df['precipitacao_mm'].std():.2f} mm")
    
    # Valores nulos
    nulos = df['precipitacao_mm'].isnull().sum()
    print(f"\n  Valores nulos: {nulos:,} ({nulos/len(df)*100:.2f}%)")
    
    # Visualização
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Histograma
    df['precipitacao_mm'].hist(bins=100, ax=axes[0], edgecolor='black', alpha=0.7)
    axes[0].set_title('Distribuição de Precipitação', fontweight='bold', fontsize=12)
    axes[0].set_xlabel('Precipitação (mm)')
    axes[0].set_ylabel('Frequência')
    axes[0].grid(alpha=0.3)
    
    # Box plot
    df['precipitacao_mm'].plot(kind='box', ax=axes[1], vert=True)
    axes[1].set_title('Box Plot de Precipitação', fontweight='bold', fontsize=12)
    axes[1].set_ylabel('Precipitação (mm)')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Dados de precipitação não disponíveis!")


Dados de precipitação não disponíveis!


## 8. Análise Temporal


In [8]:
if len(df) > 0 and 'timestamp_utc' in df.columns:
    # Extrai componentes temporais
    df['ano'] = df['timestamp_utc'].dt.year
    df['mes'] = df['timestamp_utc'].dt.month
    df['dia'] = df['timestamp_utc'].dt.day
    df['hora'] = df['timestamp_utc'].dt.hour
    
    # Agregação mensal de precipitação
    if 'precipitacao_mm' in df.columns:
        df_mensal = df.groupby(['ano', 'mes']).agg({
            'precipitacao_mm': 'sum'
        }).reset_index()
        
        # Cria data a partir de ano e mês (corrigido)
        df_mensal['data'] = pd.to_datetime({
            'year': df_mensal['ano'],
            'month': df_mensal['mes'],
            'day': 1
        })
        
        # Visualização
        plt.figure(figsize=(14, 6))
        plt.plot(df_mensal['data'], df_mensal['precipitacao_mm'], marker='o', linewidth=2, markersize=4)
        plt.title('Precipitação Mensal Total', fontsize=14, fontweight='bold')
        plt.xlabel('Data', fontsize=12)
        plt.ylabel('Precipitação Total (mm)', fontsize=12)
        plt.grid(alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        print("\nPrecipitação por Mês:")
        display(df_mensal)
else:
    print("Dados temporais não disponíveis!")


Dados temporais não disponíveis!


## 9. Resumo da Análise Exploratória


In [9]:
print("=" * 80)
print("RESUMO DA ANÁLISE EXPLORATÓRIA")
print("=" * 80)

if len(df) > 0:
    print(f"\nDataset carregado com sucesso!")
    print(f"   - Total de registros: {len(df):,}")
    print(f"   - Número de estações: {df['codigo_wmo'].nunique()}")
    print(f"   - Período: {df['timestamp_utc'].min()} até {df['timestamp_utc'].max()}")
    
    print(f"\nPróximos passos:")
    print(f"   1. Executar notebook 02_tratamento_limpeza.ipynb")
    print(f"   2. Tratar valores faltantes")
    print(f"   3. Classificar intensidade de chuva")
    print(f"   4. Preparar dados para modelagem")
else:
    print("\nNenhum dado disponível!")
    print("   Execute a ingestão de dados primeiro.")

print("\n" + "=" * 80)


RESUMO DA ANÁLISE EXPLORATÓRIA

Nenhum dado disponível!
   Execute a ingestão de dados primeiro.

